# Here be dragons.
### Before you do anything
* Read everything below.
* Download a complete, fresh copy of the repo you want to shrink.
* Copy that repo again on your computer.
* Run this carefully on a copy of the newly copied repo. Copy?

### What this does
This attempts to take the non-main/non-master branches of a Git repo that's been bloated through excessive snapshots of Github Actions. It aims to substantially shrink the size of repos.

It builds a branch of each of those original branches; commits those files; deletes the original branch; and uploads the replacement in its stead.

This in effect flattens the repo and kills off all the history, including older snapshots of working files that have been getting diff'd.

In [1]:
import sys
!{sys.executable} -m pip install GitPython

In [2]:
from git import *

In [3]:
untouchables = ["origin/HEAD", "origin/main", "origin/master"]    # What branches should never be trimmed?

In [4]:
repo = Repo('.')
assert not repo.bare

In [5]:
git = repo.git

In [6]:
remote_refs = repo.remote().refs

In [9]:
branchestotrim = {}

In [10]:
for remote_ref in remote_refs:
    if remote_ref.name not in untouchables:
        branchhost = remote_ref.name.split("/")[0]   
        branchtarget = "/".join(remote_ref.name.split("/")[1:])
        branchestotrim[remote_ref.name] = {
            "host": branchhost,
            "target": branchtarget
        }

In [11]:
# Make remote branches locally trackable

# git branch --track origin/ca#origin origin/ca
for branchtotrim in branchestotrim:
    branchhost = branchestotrim[branchtotrim]["host"]    
    branchtarget = branchestotrim[branchtotrim]["target"]
    git.branch("--track", f"{branchtotrim}#{branchhost}", branchtotrim)

In [14]:
for branchtotrim in branchestotrim:
    localname = "local-" + branchtotrim.split("/")[-1]
    branchhost = branchestotrim[branchtotrim]["host"]    
    branchtarget = branchestotrim[branchtotrim]["target"]
    git.checkout(f"{branchtotrim}#{branchhost}")
    print(f"Working on {branchtotrim} at {localname}")
    git.checkout("--orphan", localname)
    git.add("-A")     # Add all files
    changedFiles = repo.index.diff("HEAD")
    if not changedFiles:      # Delete temporary branch and do nothing else
        git.checkout(branchtotrim)
        git.branch("-D", localname)
    else:              # Files changed; replace old branch with temporary branch
        git.commit("-m", f"Cleaning up {branchtotrim}")
        git.branch("-D", branchtotrim)    # Delete old branch
        git.branch("-m", branchtotrim)    # Move temporary branch into its place
        git.push("-u", "-f", branchhost, branchtarget)    # And upload

GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git checkout origin/ak#origin
  stderr: 'error: Your local changes to the following files would be overwritten by checkout:
	trim-repo.ipynb
Please commit your changes or stash them before you switch branches.
Aborting'